<h1> BART for inequalities </h1>

In [1]:
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

<h2> Preprocessing </h2>

Life expectancy for all countries from the World Bank

In [2]:
life_exp_df = pd.read_csv("wb_life_expectancy.csv", skiprows = 4)
life_exp_df = life_exp_df[["Country Name", "2017"]]
life_exp_df.rename(columns = {"2017":"Life Expectancy 2017"}, inplace=True)
life_exp_df = life_exp_df.dropna()
life_exp_df

,Country Name,Life Expectancy 2017
0,Aruba,76.010000
1,Afghanistan,64.130000
2,Angola,60.379000
3,Albania,78.333000
5,Arab World,71.622526
...,...,...
259,Kosovo,71.946341
260,"Yemen, Rep.",66.086000
261,South Africa,63.538000
262,Zambia,63.043000


Income distribution for all countries and world regions from WID.world. The distribution is split into bottom 50 percent, 50-90 percent (middle class), top 10 percent and the top 1 percent share

In [3]:
income_df = pd.read_csv("wid_income_dist.csv", skiprows = 1, sep = ";", header = None)
income_df = income_df[[0, 2, 4]]
income_df.columns = ["Region Name", "percentile", "Income Share"]
income_df = income_df.dropna() # Only keep regions with all 4 parts of the income distribution
income_df = income_df.pivot(index='Region Name', columns='percentile')['Income Share'] # reshape, col per share
income_df

percentile,p0p50,p50p90,p90p100,p99p100
Region Name,,,,
Africa,0.088212,0.368794,0.542994,0.190221
Albania,0.209400,0.470900,0.319700,0.082100
Algeria,0.207066,0.420077,0.372856,0.097033
Angola,0.130631,0.380834,0.488535,0.151751
Austria,0.234300,0.449100,0.316600,0.092700
...,...,...,...,...
United Kingdom,0.206100,0.439300,0.354600,0.126100
Western Africa,0.116490,0.375802,0.507708,0.164721
Zambia,0.073127,0.311930,0.614943,0.230787


Merge the life expectancy and income dataframes on country

In [4]:
le_income_df = life_exp_df.merge(income_df, left_on = "Country Name", right_on = "Region Name")
le_income_df

,Country Name,Life Expectancy 2017,p0p50,p50p90,p90p100,p99p100
0,Angola,60.379000,0.130631,0.380834,0.488535,0.151751
1,Albania,78.333000,0.209400,0.470900,0.319700,0.082100
2,Austria,81.641463,0.234300,0.449100,0.316600,0.092700
3,Burundi,60.898000,0.151344,0.371082,0.477574,0.145485
4,Belgium,81.439024,0.205900,0.480100,0.313900,0.077700
...,...,...,...,...,...,...
79,Tanzania,64.479000,0.153972,0.365047,0.480980,0.161714
80,Uganda,62.516000,0.131229,0.353945,0.514826,0.168541
81,South Africa,63.538000,0.062700,0.286500,0.650800,0.192100
82,Zambia,63.043000,0.073127,0.311930,0.614943,0.230787


In [5]:
X = le_income_df[["p0p50", "p50p90", "p90p100", "p99p100"]]
y = le_income_df[["Life Expectancy 2017"]]

<h2> Random Forest implementation </h2>

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
rf_pipeline = Pipeline(steps=[("model", RandomForestRegressor(n_estimators = 100, random_state = 0))
                             ])

In [7]:
from sklearn.model_selection import cross_val_score
# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(rf_pipeline, X, y,
                              cv = 5,
                              scoring = "neg_mean_absolute_error")

print("MAE scores:\n", scores)
print("Average MAE score (across experiments):")
print(scores.mean())

MAE scores:
 [3.88485495 2.91733043 5.3309053  3.43702363 4.52441894]
Average MAE score (across experiments):
4.018906650270395


In [8]:
from sklearn.model_selection import GridSearchCV
parameters = {"model__n_estimators": np.arange(5, 85 + 1, 20)}
grid_search = GridSearchCV(rf_pipeline, parameters)
grid_search.fit(X, y)
grid_search.best_params_

{'model__n_estimators': 45}

In [9]:
rf_pipeline = Pipeline(steps=[("model", RandomForestRegressor(n_estimators = grid_search.best_params_["model__n_estimators"], random_state = 0))
                             ])
                             
scores = -1 * cross_val_score(rf_pipeline, X, y,
                              cv = 5,
                              scoring = "neg_mean_absolute_error")

print("MAE scores:\n", scores)
print("Average MAE score (across experiments):")
print(scores.mean())

MAE scores:
 [3.76447217 2.82785205 5.27005592 3.46856951 4.47042022]
Average MAE score (across experiments):
3.9602739738883015


<h2> bartpy implementation </h2>

In [32]:
le_income_df[["p99p100"]].reset_index(drop=True)

,p99p100
0,0.151751
1,0.082100
2,0.092700
3,0.145485
4,0.077700
...,...
79,0.161714
80,0.168541
81,0.192100
82,0.230787


In [10]:
X = le_income_df[["p0p50", "p50p90", "p90p100", "p90p100"]]
y = le_income_df["Life Expectancy 2017"].values

In [13]:
bart_pipeline = Pipeline(steps=[("model", SklearnModel(n_burn = 100, n_chains = 1, n_jobs = 1, n_samples = 1000, n_trees = 10))
                             ])

In [17]:
from bartpy.sklearnmodel import SklearnModel

In [15]:
parameters = {"model__n_trees": np.arange(5, 85 + 1, 20)}
grid_search = GridSearchCV(bart_pipeline, parameters)
grid_search.fit(X, y)
grid_search.best_params_

100%|██████████| 1000/1000 [00:40<00:00, 24.68it/s]


{'model__n_trees': 85}

In [16]:
bart_pipeline = Pipeline(steps=[("model", SklearnModel(n_burn = 100, n_chains = 1, n_jobs = 1, n_samples = 1000, n_trees = grid_search.best_params_["model__n_trees"]))
                               ])
scores = -1 * cross_val_score(bart_pipeline, X, y,
                              cv = 5,
                              scoring = "neg_mean_absolute_error")
                              
print("MAE scores:\n", scores)
print("Average MAE score (across experiments):")
print(scores.mean())

100%|██████████| 1000/1000 [00:59<00:00, 16.70it/s]
MAE scores:
 [3.67273052 2.56707642 5.06297515 3.98376936 5.12356429]
Average MAE score (across experiments):
4.082023148315182


<h2> Predictions </h2>

In [19]:
rf_pipeline.fit(X, y)
y_pred_rf = rf_pipeline.predict(X)
le_income_df["Random Forest LE"] = y_pred_rf

In [20]:
bart_pipeline.fit(X, y)
y_pred_bart = bart_pipeline.predict(X)
le_income_df["BART LE"] = y_pred_bart

100%|██████████| 1000/1000 [00:52<00:00, 19.22it/s]


In [23]:
le_income_df

,Country Name,Life Expectancy 2017,p0p50,p50p90,p90p100,p99p100,Random Forest LE,BART LE
0,Angola,60.379000,0.130631,0.380834,0.488535,0.151751,60.491644,62.074536
1,Albania,78.333000,0.209400,0.470900,0.319700,0.082100,78.667501,79.928005
2,Austria,81.641463,0.234300,0.449100,0.316600,0.092700,81.858699,81.453081
3,Burundi,60.898000,0.151344,0.371082,0.477574,0.145485,62.540133,64.728669
4,Belgium,81.439024,0.205900,0.480100,0.313900,0.077700,81.509322,79.392117
...,...,...,...,...,...,...,...,...
79,Tanzania,64.479000,0.153972,0.365047,0.480980,0.161714,66.102622,65.843226
80,Uganda,62.516000,0.131229,0.353945,0.514826,0.168541,65.001578,64.636357
81,South Africa,63.538000,0.062700,0.286500,0.650800,0.192100,61.426556,59.866869
82,Zambia,63.043000,0.073127,0.311930,0.614943,0.230787,62.588289,60.660164


In [41]:
bart_pipeline = Pipeline(steps=[("model", SklearnModel(n_burn = 100, n_chains = 1, n_jobs = 1, n_samples = 1000, n_trees = grid_search.best_params_["model__n_trees"], store_in_sample_predictions = True))
                               ])
bart_pipeline.fit(X, y)
y_pred_bart = bart_pipeline.predict(X)

100%|██████████| 1000/1000 [00:49<00:00, 20.12it/s]


In [44]:
bart_pipeline.predict(None)

array([62.67857274, 79.33139433, 81.31756437, 64.49738865, 79.65272567,
       62.50621538, 65.70152363, 71.26761161, 78.97088639, 62.27887631,
       59.98349042, 81.92279157, 60.32642067, 59.70482114, 63.25322567,
       78.89303249, 80.56053448, 78.89303249, 62.86381457, 81.61350869,
       75.18061033, 65.82358251, 80.19265242, 78.23896874, 65.82358251,
       79.49456581, 80.53630902, 81.03749195, 65.29346377, 80.19265242,
       62.67857274, 64.58215633, 62.11834382, 62.46024474, 78.01127968,
       79.27143189, 77.42510508, 80.78075122, 81.83018795, 78.97088639,
       64.49738865, 64.43951911, 65.80915616, 62.23231885, 77.527518  ,
       80.39924456, 79.49456581, 66.8444121 , 78.97088639, 63.92224957,
       68.2266175 , 81.33434573, 78.39629316, 60.26591058, 70.6660783 ,
       65.70152363, 62.47833895, 60.43820891, 64.37568568, 60.70631667,
       80.85832086, 80.68284546, 76.85619494, 78.09422991, 68.25522617,
       64.24619757, 66.29137863, 65.35426649, 64.58215633, 64.41

In [75]:
# Thin is 10 so for 1000 samples, we can get 100 to calculate the error
bart_pipeline["model"].data.y.unnormalize_y(bart_pipeline["model"]._prediction_samples)

array([[62.76688031, 79.08223605, 81.32790135, ..., 60.06312764,
        61.72394989, 63.06485657],
       [62.55115875, 79.13213252, 81.49939828, ..., 59.97574512,
        61.66224219, 62.78389068],
       [62.72355719, 79.79398762, 81.59893292, ..., 60.04702361,
        61.9938215 , 62.79940883],
       ...,
       [62.95811936, 79.59519071, 81.04135461, ..., 60.07972862,
        61.98874664, 62.72495288],
       [62.69632425, 79.31376382, 81.39564956, ..., 59.65835015,
        61.75512283, 62.77869095],
       [62.47087992, 79.5632348 , 81.26095032, ..., 59.76098342,
        61.85533882, 62.8561231 ]])

In [95]:
samples = bart_pipeline["model"]._prediction_samples
# should have self.prediction_samples() method but error in package where bartpy return self.prediction_samples instead of self._prediction_samples
unnormalised_samples = bart_pipeline["model"].data.y.unnormalize_y(samples)
np.std(unnormalised_samples, axis = 0)

array([0.17033517, 0.20093172, 0.21542585, 0.11859725, 0.20329531,
       0.168604  , 0.1761632 , 0.2217806 , 0.17657382, 0.17645311,
       0.25557541, 0.24427259, 0.25892102, 0.31844918, 0.20519722,
       0.23130429, 0.28033278, 0.23130429, 0.21876647, 0.18922966,
       0.24077065, 0.22888902, 0.2335799 , 0.19090243, 0.22888902,
       0.20626436, 0.21393702, 0.20802572, 0.16933264, 0.2335799 ,
       0.17033517, 0.30831607, 0.17508999, 0.16601717, 0.25729367,
       0.20683134, 0.22873208, 0.22397738, 0.20687712, 0.17657382,
       0.11859725, 0.23135403, 0.2066821 , 0.18052864, 0.20514612,
       0.22353232, 0.20626436, 0.27536869, 0.17657382, 0.20436867,
       0.23594473, 0.25719785, 0.21248913, 0.24579737, 0.20952409,
       0.1761632 , 0.2429461 , 0.25653329, 0.23611076, 0.25980042,
       0.2485891 , 0.26042453, 0.20210817, 0.25018737, 0.21256833,
       0.23139622, 0.24595949, 0.14792622, 0.30831607, 0.21537483,
       0.27063246, 0.3185973 , 0.20952409, 0.19644508, 0.19960

In [84]:
bart_pipeline["model"].data.y.unnormalize_y(np.std(bart_pipeline["model"]._prediction_samples, axis=0))

array([68.09155468, 68.12215124, 68.13664536, 68.03981676, 68.12451482,
       68.08982351, 68.09738271, 68.14300011, 68.09779333, 68.09767262,
       68.17679493, 68.16549211, 68.18014053, 68.23966869, 68.12641673,
       68.1525238 , 68.20155229, 68.1525238 , 68.13998598, 68.11044917,
       68.16199017, 68.15010853, 68.15479942, 68.11212194, 68.15010853,
       68.12748387, 68.13515654, 68.12924523, 68.09055215, 68.15479942,
       68.09155468, 68.22953559, 68.09630951, 68.08723668, 68.17851318,
       68.12805086, 68.14995159, 68.1451969 , 68.12809663, 68.09779333,
       68.03981676, 68.15257354, 68.12790161, 68.10174815, 68.12636563,
       68.14475183, 68.12748387, 68.1965882 , 68.09779333, 68.12558818,
       68.15716424, 68.17841737, 68.13370864, 68.16701689, 68.1307436 ,
       68.09738271, 68.16416562, 68.1777528 , 68.15733027, 68.18101993,
       68.16980862, 68.18164404, 68.12332769, 68.17140688, 68.13378784,
       68.15261573, 68.167179  , 68.06914574, 68.22953559, 68.13

In [14]:
scores = -1 * cross_val_score(bart_pipeline, X, y,
                              cv = 5,
                              scoring = "neg_mean_absolute_error")
                              
print("MAE scores:\n", scores)
print("Average MAE score (across experiments):")
print(scores.mean())

100%|██████████| 1000/1000 [00:05<00:00, 168.82it/s]MAE scores:
 [3.95513412 2.61100212 4.79070707 4.49624447 4.71767429]
Average MAE score (across experiments):
4.114152413541102

